In [2]:
from pathlib import Path
from copy import deepcopy
from cjio import cityjson
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
plt.close('all')
from sklearn.preprocessing import FunctionTransformer
from sklearn import cluster
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import os

In [3]:
from shapely.geometry import shape
from functools import partial
import pyproj
from shapely.ops import transform


# 定义EPSG 7415和EPSG 28992的投影坐标系
source_crs = pyproj.CRS("EPSG:7415")
target_crs = pyproj.CRS("EPSG:28992")


#source_crs  = pyproj.Proj(init='epsg:7415')
#target_crs = pyproj.Proj(init='epsg:28992')


project = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True).transform


In [11]:
import os
import json
path = "rotterdam_center"
files = os.listdir(path)

gdf = gpd.GeoDataFrame(columns=['building_id', 'surface_type', 'geometry'])


In [10]:
# convert each cityjson to geojson, NO merging

print(files)
for file in files:
    polygons = []
    surfaces = []
    building_ids = []
    if ".city.json" not in file:
        continue

    cm = cityjson.load(os.path.join(path,file),transform=True)
    
    tranformation_object = cm.transform
    tranformation_object["scale"] = [0.001,0.001,1]
    
    buildings = cm.get_cityobjects()
    for building_id,data in buildings.items():
        
        if data.type == "Building":
            continue
        #print(data.parents[0])
        #print(data.geometry[0].transform(tranformation_object).boundaries)
        
        for geom in data.geometry:
            
            boundaries = geom.boundaries
            #print(bounda
           
            
            for surface in geom.surfaces:

                idx = np.array(geom.surfaces[surface]["surface_idx"])
                surface_type = geom.surfaces[surface]["type"]
                if idx.any()==None:
                    continue
                #print([Polygon(boundaries[idx[0][0]][index[1]][0]) for index in idx])
                
                smalls = [Polygon(boundaries[index[0]][index[1]][0],boundaries[index[0]][index[1]][1:]) \
                             for index in idx]
               
                
                polygons += smalls
                surfaces += [surface_type]*len(smalls)
                building_ids += [building_id]*len(smalls)

    gdf =  gpd.GeoDataFrame({'building_id': building_ids, 'surface_type': surfaces, 'geometry': polygons},
                             crs=7415)

    gdf["geometry"].set_crs(epsg=7415)
    gdf["geometry"].to_crs(epsg=28992) 

    tile_path = file.split(".")[0] + ".geojson"
    gdf.to_file(os.path.join(path,tile_path), driver='GeoJSON')   
    print("done", tile_path)

['10-268-588.city.json', '10-268-590.city.json', '10-268-592.city.json', '10-268-594.city.json', '10-270-588.city.json', '10-270-590.city.json', '10-270-592.city.json', '10-270-594.city.json', '9-268-584.city.json', '9-272-584.city.json', '9-272-588.city.json', '9-272-592.city.json', '9-276-584.city.json', '9-276-588.city.json', '9-276-592.city.json', '9-280-584.city.json', '9-280-588.city.json', '9-280-592.city.json']
done 10-268-588.geojson
done 10-268-590.geojson
done 10-268-592.geojson
done 10-268-594.geojson
done 10-270-588.geojson
done 10-270-590.geojson
done 10-270-592.geojson
done 10-270-594.geojson
done 9-268-584.geojson
done 9-272-584.geojson
done 9-272-588.geojson
done 9-272-592.geojson
done 9-276-584.geojson
done 9-276-588.geojson
done 9-276-592.geojson
done 9-280-584.geojson
done 9-280-588.geojson
done 9-280-592.geojson


'\ngdf =  gpd.GeoDataFrame({\'building_id\': building_ids, \'surface_type\': surfaces, \'geometry\': polygons},\n                             crs=7415)\n\ngdf["geometry"].set_crs(epsg=7415)\ngdf["geometry"].to_crs(epsg=28992)\n    \ngdf.to_file(os.path.join(path,\'building_surfaces.geojson\'), driver=\'GeoJSON\')\n'

In [104]:
gdf.head(5)

,building_id,surface_type,geometry
0,NL.IMBAG.Pand.0599100000013175-0,GroundSurface,"POLYGON Z ((95185.927 437834.483 30.007, 95185..."
1,NL.IMBAG.Pand.0599100000013175-0,RoofSurface,"POLYGON Z ((95185.937 437834.478 40.335, 95185..."
2,NL.IMBAG.Pand.0599100000013175-0,WallSurface,"POLYGON Z ((95185.929 437834.487 40.335, 95185..."
3,NL.IMBAG.Pand.0599100000013175-0,WallSurface,"POLYGON Z ((95185.939 437834.482 40.335, 95185..."
4,NL.IMBAG.Pand.0599100000013175-0,WallSurface,"POLYGON Z ((95185.927 437834.483 40.335, 95185..."


In [7]:
# merge and convert each cityjson to ONE geojson

print(files)
polygons = []
surfaces = []
building_ids = []
for file in files:
    if ".city.json" not in file:
        continue
    
    print(file)
    cm = cityjson.load(os.path.join(path,file),transform=True)
    
    tranformation_object = cm.transform
    tranformation_object["scale"] = [0.001,0.001,1]
    
    buildings = cm.get_cityobjects()
    for building_id,data in buildings.items():
        
        if data.type == "Building":
            continue
        #print(data.parents[0])
        #print(data.geometry[0].transform(tranformation_object).boundaries)
        
        for geom in data.geometry:
            
            boundaries = geom.boundaries
            #print(bounda
           
            
            for surface in geom.surfaces:

                idx = np.array(geom.surfaces[surface]["surface_idx"])
                surface_type = geom.surfaces[surface]["type"]
                if idx.any()==None:
                    continue
                #print([Polygon(boundaries[idx[0][0]][index[1]][0]) for index in idx])
                
                smalls = [Polygon(boundaries[index[0]][index[1]][0],boundaries[index[0]][index[1]][1:]) \
                             for index in idx]
               
                
                polygons += smalls
                surfaces += [surface_type]*len(smalls)
                building_ids += [building_id]*len(smalls)

    
gdf =  gpd.GeoDataFrame({'building_id': building_ids, 'surface_type': surfaces, 'geometry': polygons},
                             crs=7415)

gdf["geometry"].set_crs(epsg=7415)
gdf["geometry"].to_crs(epsg=28992)
merge_folder = path + '\merge'
merged_geojson = path + "_merged_building_surfaces.geojson"
gdf.to_file(os.path.join(merge_folder,merged_geojson), driver='GeoJSON')


['10-280-560.city.json', '10-280-562.city.json', '10-282-560.city.json', '10-282-562.city.json', '10-284-560.city.json', '10-284-562.city.json', '10-286-560.city.json', '10-286-562.city.json', '8-288-552.city.json', '8-288-560.city.json', '9-280-552.city.json', '9-280-556.city.json', '9-280-564.city.json', '9-284-552.city.json', '9-284-556.city.json', '9-284-564.city.json', 'geojson', 'merge']
10-280-560.city.json
10-280-562.city.json
10-282-560.city.json
10-282-562.city.json
10-284-560.city.json
10-284-562.city.json
10-286-560.city.json
10-286-562.city.json
8-288-552.city.json
8-288-560.city.json
9-280-552.city.json
9-280-556.city.json
9-280-564.city.json
9-284-552.city.json
9-284-556.city.json
9-284-564.city.json


In [12]:
# Merge all the tiles into one CityJSON file

print(files)
# Initialize an empty CityJSON object to store the merged data
merged_cityjson = {
    "type": "CityJSON",
    "version": "1.1",  # Update with the appropriate version
    "CityObjects": {},
    # Add other metadata as needed
}

# Loop through the files in the directory
for filename in files:
    if filename.endswith('.city.json'):
        file_path = os.path.join(path, filename)
        with open(file_path, 'r') as file:
            cityjson_data = json.load(file)
    
    # Merge CityObjects
    merged_cityjson['CityObjects'].update(cityjson_data['CityObjects'])
    # You may need to handle metadata updates here as well
        
# Save the merged CityJSON to a new file
merge_folder = path + '\merge'
merged_path = os.path.join(merge_folder, path + '_merged.city.json')
with open(merged_path, 'w') as merged_file:
    #json.dump(merged_cityjson, merged_file)
    json.dump(merged_cityjson, merged_file, separators=(',', ':'))

print("Merged CityJSON file saved to", merged_path)

['8-296-512.city.json', '9-288-512.city.json', '9-288-516.city.json', '9-292-512.city.json', '9-292-516.city.json', '9-304-512.city.json', '9-304-516.city.json', '9-308-512.city.json', '9-308-516.city.json', '9-308-520.city.json', '9-312-512.city.json', '9-312-516.city.json', '9-312-520.city.json', '9-316-512.city.json', '9-316-516.city.json', '9-316-520.city.json', '9-320-512.city.json', '9-320-516.city.json', '9-320-520.city.json', '9-324-512.city.json', '9-324-516.city.json', '9-324-520.city.json', 'geojson', 'merge']
Merged CityJSON file saved to rotterdam_center\merge\rotterdam_center_merged.city.json
